#### GenAI for Webscraping

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

## Data Ingestion--From the website we need to scrape the data

In [14]:
!pip install langchain_community
!pip install beautifulsoup4
from langchain_community.document_loaders import WebBaseLoader


   ---------------------------------------- 0/2 [soupsieve]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]



In [15]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Cristiano_Ronaldo")
loader
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cristiano_Ronaldo', 'title': 'Cristiano Ronaldo - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nCristiano Ronaldo - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nClub career\n\n\n\n\nToggle Club

### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cristiano_Ronaldo', 'title': 'Cristiano Ronaldo - Wikipedia', 'language': 'en'}, page_content="Cristiano Ronaldo - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nClub career\n\n\n\n\nToggle Club career 

### Covert to text to vector using open AI

In [17]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   -------- ------------------------------- 3.9/18.9 MB 26.0 MB/s eta 0:00:01
   ------------------------------ --------- 14.4/18.9 MB 37.7 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 36.1 MB/s  0:00:00


In [18]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [19]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)
vectorstoredb

In [23]:
## Query From a vector db
query="When ronaldo did debut for portugal"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"At the age of 18, Ronaldo made his debut for Portugal as a substitute against Kazakhstan on 20 August 2003.[203] At UEFA Euro 2004, he scored his first international goal in a 2–1 group stage loss to Greece in his eighth international appearance.[204][205] Despite Portugal losing to Greece again in the final, Ronaldo made the team of the tournament, with two assists and two goals.[206] In the 2006 FIFA World Cup, at the age of 21 years and 132 days, Ronaldo became the youngest ever goalscorer for Portugal at a World Cup finals.[207] Portugal reached the semi-finals, where Portugal lost to France, with Ronaldo being booed during their defeat, due to an incident that occurred in the quarter-finals against England.[208] FIFA's Technical Study Group overlooked him for the tournament's Best Young Player award and handed it to Germany's Lukas Podolski, citing his behaviour as a factor in the decision.[209][210]"